[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/googlecolab/colabtools/blob/master/notebooks/colab-github-demo.ipynb)

In [ ]:
################# Isnstallation instructions (for running in Colab) #####################
# !pip install defaultlist http_request_randomizer selenium
# !apt-get update
# !apt install chromium-chromedriver
# !cp /usr/lib/chromium-browser/chromedriver /usr/bin

In [ ]:
################# Import Statements #####################
from http_request_randomizer.requests.proxy.requestProxy import RequestProxy
from bs4 import BeautifulSoup as bs

import re
import pandas as pd
from collections import OrderedDict
from selenium.webdriver.chrome.options import Options
from defaultlist import defaultlist
import requests
import random
import pandas as pd
from random import randint
import time
from lxml.html import fromstring
from selenium import webdriver
from os import listdir
from os.path import isfile, join

from random import randint
import random

In [ ]:
################# Generate product name URL #####################
def create_url(product_term, page=1):
    """Create a url based on search terms. """

    base_url = "https://www.amazon.com/"

    product_term = product_term.replace('dp','product-reviews')
    # Inserting query   
    search_url = base_url+product_term

    # Add provision to travel through different pages
    if page > 1:
      search_url += '/ref=cm_cr_arp_d_paging_btm_next_{}?_encoding=UTF8&pageNumber={}'.format(page,page)
    else:
      pass

    return search_url

In [ ]:
################# Prepare the webdriver #####################
def set_driver(PROXY=None):
  """This function is used to set the ChromeWebDriver with all the necessary options like Proxy."""
  chrome_options = Options()
  chrome_options.add_argument('--headless')
  chrome_options.add_argument('--no-sandbox')
  chrome_options.add_argument('--disable-dev-shm-usage')
  if PROXY==None:
    driver = webdriver.Chrome('/usr/bin/chromedriver',options=chrome_options)
  else:
    chrome_options.add_argument('--proxy-server=%s' % PROXY)
    driver = webdriver.Chrome('/usr/bin/chromedriver',options=chrome_options)
  return driver

In [ ]:
################# Scraper function #####################
def execute_scraper_amazon(driver, ind, product_term, output_path, start=1, end=10):
    """Run the scraper and display the values as required from amazon. """
    data = {}

    reviews = defaultlist(lambda: 'empty')
    loc_date = defaultlist(lambda: 'empty')
    page_number = defaultlist(lambda: 'empty')
    book = defaultlist(lambda:'empty')
    index = defaultlist(lambda:'empty')

    file_name = product_term.split('/')[0].replace('-',' ')

    #creating the query term & fetching the requested page
    url = create_url(product_term)

    driver.get(url)
    try:
      driver.set_page_load_timeout(5)
    except:
      driver.set_page_load_timeout(10)
      driver.get(url)

    page_source = driver.page_source
    soup = bs(page_source,'html.parser')

    time.sleep(randint(20,30))

    for i in range(start,end):

        if i == 0:
          index.append(ind)
          pass
        else:
          index.append(ind+i)
          url = create_url(product_term,i)

        time.sleep(randint(30,40))

        try:
          driver.get(url)
          driver.set_page_load_timeout(10)
        except:
          driver.get(url)
          driver.set_page_load_timeout(20)   
        finally:
          driver.get(url)

        page_source = driver.page_source
        soup = bs(page_source,'html.parser')

        if soup.find("span",{"data-hook":"review-body"}) == None or '':
            print('Stopped working')
            break

        else:
            print('Working')
            time.sleep(randint(30,50))
            for j,k in zip(soup.find_all("span",{"data-hook":"review-body"}),soup.find_all("span",{"data-hook":"review-date"})):

                reviews.append(j.text.strip()) 

                loc_date.append(k.text.strip())

                page_number.append(i)
                book.append(file_name)



    if len(reviews) > 1 and len(loc_date) > 1:
      data['Reviews'] = reviews
      data['Location and date'] = loc_date
      data['Page'] = page_number
      data['Book Name'] = book
      data['Index'] = index

      df=pd.DataFrame.from_dict(data,orient='index').transpose()
      df.to_csv(output_path + file_name + f' {start} to {end}'+'.csv', index=False)

      print("Data saved")
    else:
      print("No data")

In [ ]:
################# Extract data as required #####################
def get_data(product_term,ind,start=1,end=20):
    """This function is the one that performs the extraction. 
    Since scraping websites like Amazon is not easy it is advisable to 
    take breaks betweens extractions. That is why 'start' and 'end' variables
    are responsible for defining the starting and ending review page numbers.
    Post extraction each file created will have the starting and ending pages and there
    is another function which can automatically merge all the extracted csvs into 1 file and define 
    the starting and ending extracting of the whole batch."""
    proxy = []
    print(start)
    url = create_url(product_term)
    req_proxy = RequestProxy() #you may get different number of proxy when  you run this at each time
    proxies = req_proxy.get_proxy_list()
    # driver = set_driver(Options())
    for i in range(randint(1,5)):
      try:
        proxy_add = proxies[i].get_address()
        driver = set_driver(proxy_add)
        driver.get(url);
        driver.set_page_load_timeout(10);
        proxy.append(proxy_add)
      except:
        pass
    # print(proxy)
    # print(proxies)
    try:
      driver = set_driver(PROXY=proxy[-1])
      execute_scraper_amazon(driver,ind,product_term,start=start,end=end)
    except:
      driver = set_driver()
      execute_scraper_amazon(driver,ind,product_term,start=start,end=end)
    print(f"Done for {product_term} from {start} to {end}")

In [ ]:
################# Append CSVs to 1 file #####################
def stitch_csvs_together(product_term,file_name_array):
  """This funciton iteratively stitches the CSVs generated into one file."""
  merged_csvs = []
  # starting_page and ending_page variables hold the values of all the starting and ending pages in all of these files.
  starting_page = []
  ending_page = []
  file_name = product_term.split('/')[0].replace('-',' ')
  for i in file_name_array:
      extracted = re.search('[\d]{1,} to [\d]{1,}',i).group(0)
      extracted_list = extracted.split(' ')
      starting_page.append(int(extracted_list[0]))
      ending_page.append(int(extracted_list[-1]))
      csv_to_df = pd.read_csv(f"/content/{i}")
      merged_csvs.append(csv_to_df)
  output_start = min(starting_page)
  output_end = max(ending_page)
  output_file_name = file_name+' '+str(output_start)+' to '+str(output_end)
  print(output_file_name)
  merged = pd.concat(merged_csvs)
  merged.to_csv(f"{output_file_name}.csv", index=False)
  print('Files have been merged.')

In [ ]:
################# Calling all the functions one by one. #####################

#Some test URLs.
test_urls = [
"https://www.amazon.com/Hunger-Games-Book/dp/0439023483",
"https://www.amazon.com/Last-Wish-Andrzej-Sapkowski/dp/0316029181",
"https://www.amazon.com/Heard-You-Paint-Houses-Teamsters/dp/1586420771"]

test_product_terms = [i.replace("https://www.amazon.com/","") for i in test_urls]

product_term =  test_product_terms[0]
#'Hunger-Games-Book/dp/0439023483'

#Calls the get_data() function to both scrape data and compile the scraped data into a CSV file.
get_data(product_term, ind=301, start=31, end=61)

try:
  output_path = '/usr/Desktop/' + re.sub("\/dp\/[\d]{1,}", "", test_product_terms[0])
except:
  output_path = input("Please enter the complete path where files have been extracted (for each product). \
                      Suppose you have a directory in Desktop with the file name Hunger-Games-Book where \
                      reviews only for that book is being stored then you should enter - /user/Desktop/Hunger-Games-Book (where user is your user name) : ")

#The file_name_array list consists of the file names that have been generated.
# file_name_array = ['Hunger Games Book 1 to 0.csv', 'Hunger Games Book 2 to 10.csv', 'Hunger Games Book 10 to 31.csv', 'Hunger Games Book 31 to 61.csv']
file_name_array = [f for f in listdir(output_path) if isfile(join(output_path, f))]

#Calling the stitch_csvs_together() to stitch all files in file_name_array.
stitch_csvs_together(product_term,file_name_array)